# Chapter 1 The Learning Problem

## Exercises

#### Exercise 1.1

1. (a) 
  * Input space $\mathcal{X}$: patient's medical history, symptoms, personal health information etc.
  * Output space $\mathcal{Y}$: all possible diseases
  * target function $f: \mathcal{X} \to \mathcal{Y}$: ideal formula to identify a patient's problem
  * Data set: All available patients' information and their corresponding correct problem diagnostic.
2. (b)
  * Input space $\mathcal{X}$: handwritten digits (digitalized)
  * Output space $\mathcal{Y}$: 0-9 digits
  * target function $f: \mathcal{X} \to \mathcal{Y}$: ideal formula match a handwritten digit to a correct digit
  * Data set: handwritten digits and their corresponding correct matches
3. (c)
  * Input space $\mathcal{X}$: every information of an email, e.g. words
  * Output space $\mathcal{Y}$: yes/no 
  * target function $f: \mathcal{X} \to \mathcal{Y}$: ideal formula to identify whether an email is spam or not
  * Data set: Spam and non-spam emails that have been identified by human
4. (d)
  * Input space $\mathcal{X}$: price of electric, temperature, day of the week
  * Output space $\mathcal{Y}$: electric load
  * target function $f: \mathcal{X} \to \mathcal{Y}$: ideal function that gives exact electric load for a given price, temperature and day of the week.
  * Data set: historical electric load along with corresponding price, temperature and day of the week information.
5. (e) 
  * Input space $\mathcal{X}$: cat images 
  * Output space $\mathcal{Y}$: types of cats
  * target function $f: \mathcal{X} \to \mathcal{Y}$: ideal function that gives cat type according to the picture
  * Data set: picutres with cats that have been categorized to various types

#### Exercise 1.2
1. (a) Keywords with a large positive weight: free, cheap, earn, !
1. (b) Keywords with a negative weight: person name, hi, the
1. (c) The parameter $b$ in perceptron directly affects how many borderline messages end up being classified as spam. This is because $b$ is the threshold used to classify the emails into spam and non-spam categories.

#### Exercise 1.3
1. (a) If $x(t)$ is misclassified by $w(t)$, then $w^T(t)x(t)$ has different signs of $y(t)$, thus $y(t)w^T(t)x(t) \gt 0$.
2. (b) 
\begin{align}
y(t)w^T(t+1)x(t)  &= y(t) \left(w(t)+y(t)x(t)\right)^Tx(t) \\
&= y(t)\left(w^T(t) + y(t)x^T(t)\right)x(t) \\
&= y(t)w^T(t)x(t) + y(t)y(t)x^T(t)x(t)\\
&\gt y(t)w^T(t)x(t) \;\;\;\text{because the last term is } \ge \text{ than } 0\\
\end{align}
3. (c) 

From previous problem, we see that $y(t)w^T(t)x(t)$ is increasing with each update. If $y(t)$ is positive, but $w^T(t)x(t)$ is negative, we move $w^T(t)x(t)$ toward positive by increasing it. If however $y(t)$ is negative, but $w^T(t)x(t)$ is positive, $y(t)w^T(t)x(t)$ increases means $w^T(t)x(t)$ is decreasing, i.e. moving toward negative region. 

So the move from $w(t)$ to $w(t+1)$ is a move "in the right direction" as far as classifying $x(t)$ is concerned.

#### Exercise 1.4 TODO
Implement the perceptron learning algorithm and check
* Convergence speed
* How well the final hypothesis $g$ matches your target $f$

#### Exercise 1.5
1. (a) Learning
1. (b) Design
1. (c) Learning
1. (d) Design
1. (e) Learning

#### Exercise 1.6
1. (a) Supervised Learning
1. (b) Reinforcement Learning
1. (c) Unsupervised Learning
1. (d) Learning to play music
  * If learn by yourself, it's unsupervised learning
  * If learn from a teacher, it's supervised learning
  * If learn by yourself but with someone to tell you if your music is good or not, it's reinforcement learning. 
1. (e) Supervised Learning

#### Exercise 1.7
1. (a) The learning algorithm will pick the final hypothesis that always returns 1. 
  * 1 out of 8 $f$ agrees with $g$ on all three points; 3 $f$ agree with $g$ on two of the points; 3 $f$ agree with $g$ on one of the points, 1 $f$ agrees with none of the points. 
1. (b) The learning algorithm will pick the final hypothesis that always returns 0. 
  * 1 out of 8 $f$ agrees with $g$ on all three points; 3 $f$ agree with $g$ on two of the points; 3 $f$ agree with $g$ on one of the points, 1 $f$ agrees with none of the points.
1. (c) The learning algorithm will pick the final hypothesis $XOR$. 
  * 1 out of 8 $f$ agrees with $g$ on all three points; 3 $f$ agree with $g$ on two of the points; 3 $f$ agree with $g$ on one of the points, 1 $f$ agrees with none of the points.
1. (d) The learning algorithm will pick the final hypothesis $f_7$. 
  * 1 out of 8 $f$ agrees with $g$ on all three points; 3 $f$ agree with $g$ on two of the points; 3 $f$ agree with $g$ on one of the points, 1 $f$ agrees with none of the points.
  
#### Exercise 1.8

In a sample of 10 marbles, for the fraction $\nu$ of red marbles to be $\nu \le 0.1$, we must have at most one red marbles. 

\begin{align}
P(number\;of\;red \le 1) &= P(red = 0) + P(red = 1) \\
&= (1-\mu)^{10} + \mu (1-\mu)^9\\
&= (1-\mu)^9\\
&= 1.0e-9
\end{align}

#### Exercise 1.9
We have $\mu=0.9$, $N=10$, and want $\nu \le 0.1$, i.e. $|\mu - \nu| = \mu - \nu \ge 0.9 - 0.1 = 0.8$. Let's pick $\epsilon = 0.7$, then according to Hoeffding Inequity, we have

\begin{align}
P(\nu \le 0.1) &= P(\mu - \nu \ge 0.8)\\
&= P(|\mu - \nu| \ge 0.8) \\
&\le P(|\mu - \nu| \gt 0.7) \\
&= P(|\mu - \nu| \gt \epsilon) \\
&\le 2e^{-2\epsilon^2N}\\
&\approx 0.0001109032
\end{align}

This is an upper bound of the probability from pervious problem and is much larger than the calculated probability.

#### Exercise 1.10 TODO
A compute simulation exercise.

#### Exercise 1.11

1. (a) $S$ can not produce a hypothesis that is guaranteed to perform better than random on any point outside $\mathcal{D}$. 
If $f$ has 25 $+1$ on $\mathcal{D}$ but $-1$ on all other points in $\mathcal{X}$, $S$ will choose the hypothesis $h_1$, which will not match $f$ outside of $\mathcal{D}$ at all. On the other hand, a random function will have $+1$ and $-1$ 50/50, and it matches $f$ half of time, which is better than the function produced by $S$.

1. (b) It is possible that $C$ produces a better hypothesis than $S$ produces. See the example above.
1. (c) If every point in $\mathcal{D}$ has 1, then $S$ will choose $h_1$ and $C$ will choose $h_2$. So outside of $\mathcal{D}$, $h_1$ will have 90% chance to match with $f$, while $h_2$ will have only 10% chance. $S$ will always produce a better hypothesis than $C$. 

1. (d) From previous problem, we can see that when $p \lt 0.5$, $C$ will produce a better hypothesis than $S$. Since $C$ always produce $h_2$, which will match $f$ better than $h_1$ if $p \lt 0.5$.

#### Exercise 1.12

I think the best I can promise is (c). 
* The unknown target $f$ can be very complex that we can't learn at all.
* If we can learn and produce a hypothesis $g$, since there are many data points (4000), the probability that $g$ matches $f$ is high according to Hoeffding inequality, and the error on $g$ might be small since we have a large data set.

#### Exercise 1.13

1. (a) The probability of error that $h$ makes in approximating $y$ is

\begin{align}
P(h \ne y) &= P(h \ne y| y = f(x))P(y=f(x)) + P(h\ne y| y \ne f(x))P(y\ne f(x))\\
&= \mu \lambda + (1-\mu)(1-\lambda)\\
&= \mu (2 \lambda - 1) + (1-\lambda)\\
\end{align}

1. (b) It can be seen from previous problem, that when $\lambda = 0.5$, $P(h\ne y) = 1-\lambda = 0.5$, is independent of $\mu$.

## Problems

#### Problem 1.1

\begin{align}
P(1st \;ball \;is\; black) &= P(1st\; ball\; is\; black|picked\; black,black\; bag)P(picked\; black,black\; bag) + P(1st\; ball\; is\; black|picked\; black,white\; bag)P(picked\; black,white\; bag)\\
&= 1.0*0.5 + 0.5 * 0.5\\
&= 0.75\\
\end{align}

\begin{align}
P(picked\;black,black \;bag|1st\; ball\; is\; black) &= \frac{P(picked\; black,black\; bag\; and\; 1st\; ball\; is\; black)}{P(1st\; ball\; is\; black)} \\
&= \frac{P(1st\; ball\; is\; black|picked\; black,black\; bag)P(picked\; black,black\; bag)}{P(1st \;ball \;is\; black)}\\
&= \frac{1.0*0.5}{0.75}\\
&= \frac{2}{3}\\
\end{align}

\begin{align}
P(picked\;black,white \;bag|1st\; ball\; is\; black) &= \frac{1}{3}\\
\end{align}

Now we can compute the probability that the second ball is also black:

\begin{align}
P(2nd\; ball\; is \;black) &= P(2nd\; ball\; is\; black|picked\;black,black \;bag|1st\; ball\; is\; black)*P(picked\;black,black \;bag|1st\; ball\; is\; black) + P(2nd\; ball\; is \;black|picked\;black,white \;bag|1st\; ball\; is\; black)*P(picked\;black,white \;bag|1st\; ball\; is\; black)\\
&= 1.0*\frac{2}{3} + 0*\frac{1}{3}\\
&= \frac{2}{3}
\end{align}

#### Problem 1.2

1. (a)

for $h(x) =1$, we need to have $w^Tx \gt 0$, and for $h(x) = -1$, we need to have $w^Tx \lt 0$. The line separate the two regions has $w^Tx = w_0 + w_1 x_1 + w_2 x_2 = 0$, compare with $x_2 = ax_1 + b$, we have $ a = -\frac{w_1}{w_2}$ and $b =  -\frac{w_0}{w_2}$.

1. (b) The pictures for the cases $w=[1,2,3]^T$ and $w=-[1,2,3]^T$ should be be same according to the formula above.

#### Problem 1.3


1. (a) If $w^{*}$ is the optimal set of weights that separate the data, then for every $x_n$ in the data set $\mathcal{D}$, $w^{*T}x_n$ has the same sign as $y_n$, which means $y_n(w^{*T}x_n) \gt 0$. This implies $\rho = min_{1\le n\le N}y_n(w^{*T}x_n) \gt 0$.


1. (a) Let's use induction to prove this.
  * When $t=1$, we have $w^T(t-1)w^* + \rho = w^T(0)w^* + \rho = \rho$, while $w(t)=w(1) = w(0) + y_ix_i = y_ix_i$ for some $i$. So $w^T(t)w^* = (y_ix_i)^Tw^* = y_ix_i^Tw^* = y_iw^{*T}x_i$. Here we use the fact that $y_i \in {-1,1}$ and $x_i^Tw^*$ is a scalar. By definition of $\rho$, we have $y_iw^{*T}x_i \ge \rho$. Thus the inequation is valid at $t=1$.
  * Suppose the inequation is valid at $t$, let's prove that it's still valid at $t+1$. 
  $w(t+1)=w(t)+y_ix_i$ for som e $i$. Then $w^T (t+1)w^* = w^T(t)w^* + y_ix_i^Tw^* \ge w^T(t-1)w^* + \rho + y_ix_i^Tw^*  = \left(w^T(t-1) + y_ix_i^T\right)w^* + \rho = w^T(t)w^* + \rho$.
  This finishes the proof.
  
  It's easy to see that $w^T(t)w^* \ge w^T(t-1)w^* + \rho \ge w^T(t-2)w^* + 2\rho \ge \dots \ge t\rho$.
  
  
1. (c) $w(t) = w(t-1) + y(t-1)x(t-1)$, take squared norm on $w(t)$ we have $||w(t)||^2 = \left(w(t-1) + y(t-1)x(t-1)\right)^T\left(w(t-1) + y(t-1)x(t-1)\right) = ||w(t-1)||^2 + ||y(t-1)||^2||x(t-1)||^2 + 2y(t-1)w^T(t-1)x(t-1) = ||w(t-1)||^2 + ||x(t-1)||^2 + 2y(t-1)w^T(t-1)x(t-1) \le ||w(t-1)||^2 + ||x(t-1)||^2$ since $y(t-1)w^T(t-1)x(t-1) \le 0$ because $x(t-1)$ was misclassified by $w(t-1)$.

1. (d) Let's use induction.
  * When $t=0$, it's obvious that $||w(0)||^2 = 0 \le tR^2 = 0$.
  * Assume at $t$, we have $||w(t)||^2 \le tR^2$, then at $t+1$, we have $||w(t)||^2 \le ||w(t-1)||^2 + ||x(t-1)||^2 $ from previous problem. Then we have
  $||w(t)||^2 \le ||w(t-1)||^2 + ||x(t-1)||^2 \le (t-1)R^2 + R^2 = tR^2$ where we used the fact that $||x(t-1)||^2 \le R^2$ by definition of $R$.
  
1. (e) Apply the results from (b) and (d) we have $\frac{w^T(t)}{||w(t)||} w^* \ge \frac{t\rho}{||w(t)||} \ge  \frac{t\rho}{\sqrt{t}R} = \sqrt{t}\frac{\rho}{R}$.

Reverse the inequality and square, we have $t \le \frac{R^2}{\rho^2} \frac{\left(w^T(t)w^*\right)^T\left(w^T(t)w^*\right)}{||w(t)||^2} = \frac{R^2}{\rho^2} \frac{w^{*T}w(t)w^T(t)w^*}{||w(t)||^2} = \frac{R^2}{\rho^2} ||w^*||^2 $

Or we can use the hint, where $\frac{w^{*T}w^*}{||w(t)||||w^*||} \le 1$ because $||w^T(t)w^*|| \le ||w^T(t)||\cdot||w^*||$ according to the Cauchy-Schwarz inequality. Then we have $\sqrt{t}\frac{\rho}{R} \le \frac{w^{*T}w^*}{||w(t)||} \le ||w^*||$.
Move $t$ to the right and square, we obtain the desired inequality.

#### Problem 1.6

1. (a) $P(\nu=0) = (1-\mu)^{10}$, so 
  * $P(\nu = 0) =  0.5987369392383787$ when $\mu = 0.05$
  * $P(\nu = 0) =  0.0009765625$ when $\mu = 0.5$
  * $P(\nu = 0) =  1.0239999999999978e-07$ when $\mu = 0.8$


1. (b) We compute the probability that none of the 1000 samples have $\nu = 0$, i.e. all samples have $\nu \gt 0$. For a given sample, we have $P(\nu \gt 0) = 1-P(\nu = 0) = 1-(1-\mu)^{10}$.  
For all 1000 samples to have $\nu \gt 0$, the probability is 

$P(\cap_{i=1}^{1000} \nu_{i} \gt 0) = \prod_{i=1}^{1000}P(\nu_i \gt 0) = \prod_{i=1}^{1000}[1-(1-\mu)^{10}] =[1-(1-\mu)^{10}]^{1000}$

So the probability that at least one of the samples has $\nu = 0$ is: $1-P(\cap_{i=1}^{1000} \nu_{i} \gt 0) = 1-[1-(1-\mu)^{10}]^{1000}$

  * $P = 1$ when $\mu = 0.05$
  * $P = 0.623576201943276$ when $\mu = 0.5$
  * $P = 0.00010239476257623004$ when $\mu = 0.8$

1. (c) For 1000000 samples, we have the probability: $1-P(\cap_{i=1}^{1000000} \nu_{i} \gt 0) = 1-[1-(1-\mu)^{10}]^{1000000}$

  * $P = 1$ when $\mu = 0.05$
  * $P = 1$ when $\mu = 0.5$
  * $P = 0.09733159268316072$ when $\mu = 0.8$

According to Hoeffding inequality, for given tolerance, the upper bound decreases with the number of samples. The probability of the deviation between $\mu$ and $\nu'$ (proportion of red marbles in samples) becomes smaller and smaller. So $\nu'$ is closer to $\mu$.

#### Problem 1.7

1. (a) Suppose there are $m$ coins, we compute the probability that none of the coins have $\nu = 0$, i.e. all coins have $\nu \gt 0$. For a given coin, we have $P(\nu \gt 0) = 1-P(\nu = 0) = 1-P[0|N,\mu] = 1-(1-\mu)^N$.  
For all $m$ coins to have $\nu \gt 0$, the probability is 

$P(\cap_{i=1}^{m} \nu_{i} \gt 0) = \prod_{i=1}^{m}P(\nu_i \gt 0) = \prod_{i=1}^{m}[1-(1-\mu)^{N}] =[1-(1-\mu)^{N}]^{m}$

So the probability that at least one of the coins has $\nu = 0$ is: $1-P(\cap_{i=1}^{m} \nu_{i} \gt 0) = 1-[1-(1-\mu)^{N}]^{m}$

  * $P= 0.5987369392383787$ when $\mu = 0.05, m = 1$
  * $P= 1$ when $\mu = 0.05, m = 1000$
  * $P= 1$ when $\mu = 0.05, m = 1000000$
  
  * $P= 1.02400000034919e-07$ when $\mu = 0.8, m = 1$
  * $P= 0.00010239476257623004$ when $\mu = 0.8, m = 1000$
  * $P= 0.09733159268316072$ when $\mu = 0.8, m = 1000000$  
  
1. (b) TODO

#### Problem 1.8 

1. (a) Consider random variable $I(t\ge \alpha)$, which equals to $1$ when $t\ge \alpha$, else $0$. It's clear that for $\alpha \gt 0$ and $t$ a non-negative random variable, we have $\alpha I(t \ge \alpha) \le t$, take expectation on both sides, and since expectation is a monotonically increasing function here $(t \ge 0)$ , we have 
\begin{align}
E[\alpha I(t \ge \alpha)] &\le E[t]\\
\alpha E[I(t \ge \alpha)] &\le E[t]\\
E[I(t \ge \alpha)] &\le \frac{E[t]}{\alpha}\\
P[t \ge \alpha] &\le \frac{E[t]}{\alpha}\\
\end{align}

1. (b) Since $(u-\mu)^2 \ge 0$, according to problem (a) above, for any $\alpha \gt 0$, we have $P[(u - \mu)^2 \ge \alpha] \le \frac{E[(u-\mu)^2]}{\alpha} = \frac{\sigma^2}{\alpha}$.

1. (c) $u = \frac{1}{N}\sum_{n=1}^{N}$, this random variable has mean $\mu$ and variance $\frac{\sigma^2}{N}$. Take this into problem (b)'s inequation, we have $P[(u-\mu)^2 \ge \alpha] \le \frac{\sigma^2}{N\alpha}$.

This is the Chebyshev Inequality. Notice that the RHS goes down linearly in $N$, while the counterpart in Hoeffding's Inequality goes down exponentially. 

#### Problem 1.9

1. (a)   

In [19]:
mu = 0.5
m = 2
N = 6
1 - (1-(1-mu)**N)**m

0.031005859375